In [ ]:
import json
from pathlib import Path

In [ ]:
def read_json(file_path):
    """
    Reads a JSON file and returns its content.

    :param file_path: Path to the JSON file.
    :return: Parsed JSON content.
    """
    with open(file_path, "r") as file:
        return json.load(file)


def write_json(data, file_path):
    """
    Writes data to a JSON file.

    :param data: Data to write to the JSON file.
    :param file_path: Path where the JSON file will be saved.
    """
    with open(file_path, "w") as file:
        json.dump(data, file, indent=4)

In [ ]:
def extract_year(era):
    return era[:4]

In [ ]:
def subtract_lumi_lists(list_a, list_b):
    """
    Subtract lumi list B from lumi list A (A - B).
    Returns ranges that are in A but not in B.

    Args:
        list_a: Dict with run numbers as keys and list of [start, end] ranges as values
        list_b: Dict with run numbers as keys and list of [start, end] ranges as values

    Returns:
        Dict in the same format containing ranges in A but not in B
    """
    result = {}

    for run_num, ranges_a in list_a.items():
        if run_num not in list_b:
            # If run doesn't exist in B, keep all ranges from A
            result[run_num] = ranges_a[:]
        else:
            ranges_b = list_b[run_num]
            remaining_ranges = []

            for start_a, end_a in ranges_a:
                # Start with the current range from A
                current_ranges = [(start_a, end_a)]

                # Subtract each range from B
                for start_b, end_b in ranges_b:
                    new_ranges = []

                    for start_curr, end_curr in current_ranges:
                        # Check if ranges overlap
                        if end_curr < start_b or start_curr > end_b:
                            # No overlap, keep the range
                            new_ranges.append((start_curr, end_curr))
                        else:
                            # There's overlap, split the range
                            # Keep part before B's range
                            if start_curr < start_b:
                                new_ranges.append((start_curr, start_b - 1))

                            # Keep part after B's range
                            if end_curr > end_b:
                                new_ranges.append((end_b + 1, end_curr))

                    current_ranges = new_ranges

                # Add any remaining ranges
                remaining_ranges.extend(current_ranges)

            # Convert back to list format and add to result if not empty
            if remaining_ranges:
                result[run_num] = [[start, end] for start, end in remaining_ranges]

    return result

In [ ]:
def intersect_lumi_lists(list_a, list_b):
    """
    Find intersection of lumi list A and lumi list B (A ∩ B).
    Returns ranges that are in both A and B.

    Args:
        list_a: Dict with run numbers as keys and list of [start, end] ranges as values
        list_b: Dict with run numbers as keys and list of [start, end] ranges as values

    Returns:
        Dict in the same format containing ranges in both A and B
    """
    result = {}

    for run_num, ranges_a in list_a.items():
        if run_num in list_b:
            ranges_b = list_b[run_num]
            intersections = []

            for start_a, end_a in ranges_a:
                for start_b, end_b in ranges_b:
                    # Find intersection of the two ranges
                    start_intersect = max(start_a, start_b)
                    end_intersect = min(end_a, end_b)

                    # If there's a valid intersection, add it
                    if start_intersect <= end_intersect:
                        intersections.append([start_intersect, end_intersect])

            # Add to result if there are intersections
            if intersections:
                result[run_num] = intersections

    return result

In [ ]:
def count_total_lumi(lumi_dict):
    """
    Counts the total luminosity from a lumi dictionary.

    Args:
        lumi_dict: Dict with run numbers as keys and list of [start, end] ranges as values

    Returns:
        Total luminosity as an integer
    """
    total_lumi = 0
    for ranges in lumi_dict.values():
        for start, end in ranges:
            total_lumi += end - start + 1
    return total_lumi

In [ ]:
lumi_dir = Path("nano_v14_25v2_JetMET")
lumi_block_dir = lumi_dir / "lumi_blocks"
eras = ["2022", "2022EE", "2023", "2023BPix"]

lumi_lists = {era: read_json(lumi_block_dir / f"nano_v14_25v2_JetMET_{era}.json") for era in eras}

In [ ]:
golden_json_dir = Path("LumiJSON")
golden_jsons = {
    "2022": read_json(golden_json_dir / "Cert_Collisions2022_355100_362760_Golden.json"),
    "2023": read_json(golden_json_dir / "Cert_Collisions2023_366442_370790_Golden.json"),
}

In [ ]:
missing_lumis = {}
intersected_lumis = {}
for era, lumi_list in lumi_lists.items():
    intersected_lumis[era] = intersect_lumi_lists(golden_jsons[extract_year(era)], lumi_list)

In [ ]:
intersected_lumis_dir = lumi_dir / "lumi_intersections"
intersected_lumis_dir.mkdir(parents=True, exist_ok=True)
for era, lumi in intersected_lumis.items():
    write_json(lumi, intersected_lumis_dir / f"nano_v14_25v2_JetMET_{era}.json")

Then, run the following commands under `lumi_intersections` on **lxplus**
```bash
source /cvmfs/cms-bril.cern.ch/cms-lumi-pog/brilws-docker/brilws-env;

for eras in "2022" "2022EE" "2023" "2023BPix"; do brilcalc lumi --normtag /cvmfs/cms-bril.cern.ch/cms-lumi-pog/Normtags/normtag_PHYSICS.json -u /fb -i nano_v14_25v2_JetMET_${year}.json -o lumi_nano_v14_25v2_JetMET_${year}.csv; done
```